<a href="https://colab.research.google.com/github/tlaalsqh7414/pytorch_pr/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
wine = load_wine()
wine

{'DESCR': '.. _wine_dataset:\n\nWine recognition dataset\n------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 178 (50 in each of three classes)\n    :Number of Attributes: 13 numeric, predictive attributes and the class\n    :Attribute Information:\n \t\t- Alcohol\n \t\t- Malic acid\n \t\t- Ash\n\t\t- Alcalinity of ash  \n \t\t- Magnesium\n\t\t- Total phenols\n \t\t- Flavanoids\n \t\t- Nonflavanoid phenols\n \t\t- Proanthocyanins\n\t\t- Color intensity\n \t\t- Hue\n \t\t- OD280/OD315 of diluted wines\n \t\t- Proline\n\n    - class:\n            - class_0\n            - class_1\n            - class_2\n\t\t\n    :Summary Statistics:\n    \n    ============================= ==== ===== ======= =====\n                                   Min   Max   Mean     SD\n    ============================= ==== ===== ======= =====\n    Alcohol:                      11.0  14.8    13.0   0.8\n    Malic Acid:                   0.74  5.80    2.34  1.12\n    Ash:           

In [ ]:
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [ ]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [ ]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size = 0.2)

print(len(train_X), len(test_X))

104 26


In [ ]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

print(train_X.shape, train_Y.shape)

torch.Size([104, 13]) torch.Size([104])


In [ ]:
train = TensorDataset(train_X, train_Y)

print(train[0])

train_loader = DataLoader(train, batch_size = 16, shuffle = True)

(tensor([1.2370e+01, 9.4000e-01, 1.3600e+00, 1.0600e+01, 8.8000e+01, 1.9800e+00,
        5.7000e-01, 2.8000e-01, 4.2000e-01, 1.9500e+00, 1.0500e+00, 1.8200e+00,
        5.2000e+02]), tensor(1))


In [ ]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.fc1 = nn.Linear(13, 96) # input layer 노드:13   // hidden layaer 노드 : 96
    self.fc2 = nn.Linear(96, 2)

  def forward(self, x):
    x = F.relu(self.fc1(x))   # x = (sample 수) x 13 (attributes 수)
    x = self.fc2(x)           
    return F.log_softmax(x)

model = MyModel()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

for epoch in range(300):
  #total_loss = 0
  for train_x, train_y in train_loader:
    train_x, train_y = Variable(train_x), Variable(train_y)
    
    # init grad
    optimizer.zero_grad()

    # forward
    output = model(train_x)

    # 오차 계산
    loss = criterion(output, train_y)

    # backpropagation
    loss.backward()

    # update w
    optimizer.step()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


In [44]:
test_x, test_y = Variable(test_X), Variable(test_Y)

#[max(test_y[0],test_y[1]), maxIdx(0 or 1)]
result = torch.max(model(test_x).data, dim=1)[0]
print(model(test_x).data)
print(result)

acc = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

print(acc)

tensor([[-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168],
        [-0.7758, -0.6168]])
tensor([-0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168,
        -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168,
        -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168, -0.6168,
        -0.6168, -0.6168])
0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
